In [32]:
import cv2, os, pyautogui, time
import matplotlib.pyplot as plt
from src.tools import *

width, height= pyautogui.size()

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
try:
    while(True): 
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

In [94]:
# cv2.startWindowThread()
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
image = np.zeros((height, width, 3), np.uint8)
image[:,:,:] = 215
x, y = np.random.rand(), np.random.rand()
cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
cv2.imshow("window", image)
cv2.waitKey(2000)
cv2.destroyAllWindows()

# Collect

In [143]:
X, Y = [], []

## Method 1

In [134]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
try:
    for i in range(100): 
        image = np.zeros((height, width, 3), np.uint8)
        image[:,:,:] = 215
        x, y = np.random.uniform(), np.random.uniform()
        Y.append([x,y])
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)
        cv2.waitKey(500)
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

## Method 2

In [144]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)

x, y = 0.5, 0.5
try:
    for i in range(2000): 
        image = np.zeros((height, width, 3), np.uint8)
        image[:,:,:] = 215
        x = min(1, max(0, x + (np.random.uniform()-0.5)*0.01))
        y = min(1, max(0, y + (np.random.uniform()-0.5)*0.01))
        Y.append([x,y])
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)
        cv2.waitKey(20)
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

In [135]:
X = np.array(X)/255.0
Y = np.array(Y[:len(X)])

print(X.shape, Y.shape)

(105, 32, 64, 3) (105, 2)


# Train model

In [136]:
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary() 

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 30, 62, 32)        896       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 15, 31, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 6, 14, 64)         0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 4, 12, 64)         36928     
_________________________________________________________________
flatten_10 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)              

In [137]:
model.compile(optimizer='adam', loss="mse")

In [138]:
history = model.fit(X[:90], Y[:90], epochs=100, validation_data=(X[-10:], Y[-10:]))

Epoch 1/100
3/3 [==============================] - 0s 43ms/step - loss: 0.0764 - val_loss: 0.0743
Epoch 2/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0747 - val_loss: 0.0748
Epoch 3/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0748 - val_loss: 0.0754
Epoch 4/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0746 - val_loss: 0.0747
Epoch 5/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0743 - val_loss: 0.0747
Epoch 6/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0749 - val_loss: 0.0743
Epoch 7/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0738 - val_loss: 0.0742
Epoch 8/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0748 - val_loss: 0.0753
Epoch 9/100
3/3 [==============================] - 0s 20ms/step - loss: 0.0742 - val_loss: 0.0743
Epoch 10/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0736 - val_loss: 0.0741
Epoch 11/100
3/3 [=

Epoch 84/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0693 - val_loss: 0.0745
Epoch 85/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0693 - val_loss: 0.0779
Epoch 86/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0691 - val_loss: 0.0745
Epoch 87/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0694 - val_loss: 0.0721
Epoch 88/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0689 - val_loss: 0.0758
Epoch 89/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0693 - val_loss: 0.0776
Epoch 90/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0688 - val_loss: 0.0734
Epoch 91/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0686 - val_loss: 0.0756
Epoch 92/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0685 - val_loss: 0.0756
Epoch 93/100
3/3 [==============================] - 0s 22ms/step - loss: 0.0683 - val_loss: 0.0739
Epoch 94/1

# Test

In [128]:
model.predict(X[-10:])

array([[0.27037165, 0.6499416 ],
       [0.26794305, 0.8172035 ],
       [0.7593219 , 0.7707714 ],
       [0.8990095 , 0.42748293],
       [0.92324394, 0.37990242],
       [0.1450885 , 0.72960037],
       [0.46610597, 0.3824025 ],
       [0.4727258 , 0.5357595 ],
       [0.42496645, 0.82770765],
       [0.43345067, 0.34198594]], dtype=float32)

In [127]:
x, y

(0.0, 1.0)

In [125]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
try:
    while(True): 
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        [x, y] = model.predict(np.expand_dims(warpBox(frame, box, target_height=32, target_width=64), 0))[0]
        image = np.zeros((height, width, 3), np.uint8)
        image[:,:,:] = 215
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()